In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', dtype={'article_id': str})
df_sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
df['sales_channel_id'] = df['sales_channel_id'].astype('int8')
meta_art = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', dtype={'article_id': str})
meta_art.drop([
               'graphical_appearance_no','product_type_no','colour_group_code',\
               'index_code','index_group_no','product_code',\
               'perceived_colour_value_id','perceived_colour_master_id',\
               'department_no','section_no','garment_group_no'],\
              axis=1,inplace=True)
meta_art.head()

In [ ]:
# we create a sample of customers who ordered during the last week of data
# The goal is to have enough data to avoid overfitting as much as possible
customer_list = random.sample(set(df[df['t_dat']>'2020-09-14']['customer_id']),\
                              k=10000)

In [ ]:
# we create the dataframe combining transaction and articles by customer
df_info = df.merge(meta_art,how='left',on='article_id')
# we create the df to estimate some data in the 2 weeks before the last one
df_2_weeks = df_info[(df_info['t_dat']>'2020-09-01')&(df_info['t_dat']<='2020-09-14')]
# we create a sub dataframe restricted to our sample to speed up calcs
df_train = df_info[df_info['customer_id'].isin(customer_list)]
del df_info
# we create the verification dataframe
df_temp_Y = df[df['customer_id'].isin(customer_list)]
df_temp_Y = df_temp_Y[df_temp_Y['t_dat'] > '2020-09-14']

In [ ]:
# here are my ideas of variables that could help reducing the size of the candidate articles
sample_train_df = pd.DataFrame(index=customer_list)
sample_train_df['customer_id']=sample_train_df.index
sample_train_df['colour_bought'] = df_train.groupby('customer_id')\
['perceived_colour_master_name'].unique()
sample_train_df['type_of_clothing'] = df_train.groupby('customer_id')\
['product_group_name'].unique()
sample_train_df['appearance'] = df_train.groupby('customer_id')\
['graphical_appearance_name'].unique()
sample_train_df['max_budget'] = df_train.groupby('customer_id')['price'].max()
sample_train_df.head()

In [ ]:
# this function is meant to take a list of article and remove those that do not have
# a colour previously bought by the customer
# it returns the intersection of article bought during the last week and the newly created list
# it also returns the number of different bought articles during the last week
# it also returns the number of articles in the newly created list
def art_sel_colour(cust): 
    actually_bought = df_temp_Y[df_temp_Y['customer_id']==cust]['article_id']    
    temp_list = wide_list_popular_item_df
    # we remove articles with perceived colour never bought by the customer
    temp_list = set(temp_list.index).intersection(set(meta_art_pop[meta_art_pop['perceived_colour_master_name'].isin(set(sample_train_df.loc[cust,'colour_bought']))]['article_id'])) 
    temp_list1 = temp_list.intersection(set(actually_bought)) 
    return(len(temp_list1),len(set(actually_bought)),len(temp_list))

In [ ]:
# we define the list of most popular items with a certain threshold to
# reach a certain percentage of sales
# we try different thresholds in th_list and store the results in count1,2,3
th_list = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
count1 = []
count2 = []
count3 = []
for th in th_list:
  nb = (df_2_weeks['article_id'].value_counts(normalize=True).cumsum()<th).sum() 
  wide_list_popular_items = df_2_weeks['article_id'].value_counts(
      normalize=True).head(nb).index.tolist()
  wide_list_popular_item_df = pd.DataFrame(index=wide_list_popular_items)
  # we calculate the estimated price of each of these items
  # (sales can change price)
  # we create a subdataframe restricted to the popular item defined by th
  meta_art_pop = meta_art[meta_art['article_id'].isin(wide_list_popular_items)]
  # we calculate c1 the number of articles bought and in our restricted
  # list defined in article_selection for all customers we divide c1 by the 
  # number of purchases to define the maximal precision if we chose the
  # articles in our restricted list we calculate c2 the number of articles
  # in our restricted list on average we calculate c3 which is c1/c2 the
  # required 'precision' to find the good articles in our list  
  sample_train_df['possible_good_guess'] = sample_train_df['customer_id'].apply(
      art_sel_colour)  
  c1 = 0
  c2 = 0  
  c3 = 0
  for i in sample_train_df['possible_good_guess']:    
    c1 += i[0]
    c2 += i[2]    
    c3 += i[1]       
  c2 = c2/len(sample_train_df['customer_id'])  
  count1.append(c1/c3)
  count2.append(c2)
  count3.append((1-c1/c3)*len(df_temp_Y['article_id'])/(len(df_temp_Y['article_id'])-c2)) 
  # count1 is max precision that we can expect if the algorithm has 100% precision
  # count2 is the number of articles per customer on average
  # count3 is a fight between two terms (1-c1/len(df_temp_Y['customer_id'])) the loss of maximal potential precision,
  # and (len(df_temp_Y['article_id'])-c2))/len(df_temp_Y['article_id']) the loss of candidates in percentage
  # so we ideally want count3 to be as close to 0 as possible while also having c2 as low as possible
  # the work here was done on only 2 variables popularity and perceived colour, but we can imagine chaining reductions

In [ ]:
sns.relplot(x=th_list,y=count1)

In [ ]:
sns.relplot(x=th_list,y=count2)

In [ ]:
sns.relplot(x=th_list,y=count3)

In [ ]:
count1

In [ ]:
count2

In [ ]:
count3